In [1]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")

In [2]:
import numpy as np
import pandas as pd
import penaltyblog as pb
from scipy.optimize import minimize
from scipy.stats import poisson

In [3]:
df = pb.footballdata.fetch_data("England", 2018, 0)
df[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"]].head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,2018-08-10,Man United,Leicester,2,1
1,2018-08-11,Bournemouth,Cardiff,2,0
2,2018-08-11,Fulham,Crystal Palace,0,2
3,2018-08-11,Huddersfield,Chelsea,0,3
4,2018-08-11,Newcastle,Tottenham,1,2


### Poisson Model

In [4]:
pois = pb.PoissonGoalsModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

In [5]:
pois.fit()

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [6]:
pois

Module: Penaltyblog

Model: Poisson

Number of parameters: 42
Log Likelihood: -1065.077
AIC: 2214.154

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.362                -1.078              
Bournemouth          1.115                -0.777              
Brighton             0.634                -0.952              
Burnley              0.894                -0.817              
Cardiff              0.614                -0.813              
Chelsea              1.202                -1.357              
Crystal Palace       1.004                -1.061              
Everton              1.055                -1.2                
Fulham               0.626                -0.652              
Huddersfield         0.184                -0.728              
Leicester            1.0                  -1.16               
Liverpool            1.532                -1.905              
Man City         

### Dixon and Coles

In [7]:
dc = pb.DixonColesGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

In [8]:
dc.fit()

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
dc

Module: Penaltyblog

Model: Dixon and Coles

Number of parameters: 43
Log Likelihood: -1064.943
AIC: 2215.886

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.36                 -0.958              
Bournemouth          1.115                -0.655              
Brighton             0.632                -0.833              
Burnley              0.897                -0.692              
Cardiff              0.615                -0.691              
Chelsea              1.205                -1.23               
Crystal Palace       1.007                -0.937              
Everton              1.054                -1.077              
Fulham               0.625                -0.533              
Huddersfield         0.18                 -0.607              
Leicester            0.996                -1.04               
Liverpool            1.534                -1.779              
Man City 

### Rue Salvesen Model

In [10]:
rs = pb.RueSalvesenGoalModel(df["FTHG"], df["FTAG"], df["HomeTeam"], df["AwayTeam"])

In [11]:
rs.fit()

/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.9/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
rs

Module: Penaltyblog

Model: Rue Salvesen

Number of parameters: 44
Log Likelihood: -1061.167
AIC: 2210.334

Team                 Attack               Defence             
------------------------------------------------------------
Arsenal              1.312                -0.94               
Bournemouth          1.059                -0.63               
Brighton             0.664                -0.895              
Burnley              0.875                -0.701              
Cardiff              0.629                -0.737              
Chelsea              1.216                -1.298              
Crystal Palace       1.002                -0.963              
Everton              1.064                -1.141              
Fulham               0.618                -0.556              
Huddersfield         0.243                -0.714              
Leicester            1.007                -1.081              
Liverpool            1.571                -1.847              
Man City    

### Prediction

In [14]:
probs = rs.predict("Liverpool", "Leicester")

In [15]:
probs.total_goals("over", 2.5)

0.5369271457805791

In [16]:
probs.total_goals("under", 2.5)

0.4630728336858418

In [17]:
probs.asian_handicap("away", -1.5)

0.47390434527269804

In [18]:
probs.asian_handicap("home", 1.5)

0.5260956341937232

In [19]:
from pprint import pprint
params = rs.get_params()
pprint(params)

{'attack_Arsenal': 1.3120016760793092,
 'attack_Bournemouth': 1.0588195509494989,
 'attack_Brighton': 0.663664325111337,
 'attack_Burnley': 0.8750424744288547,
 'attack_Cardiff': 0.6294811036701214,
 'attack_Chelsea': 1.215858639718227,
 'attack_Crystal Palace': 1.0023366877127713,
 'attack_Everton': 1.0641132668379063,
 'attack_Fulham': 0.6176359248987324,
 'attack_Huddersfield': 0.24252036707099134,
 'attack_Leicester': 1.0070419542645253,
 'attack_Liverpool': 1.5709862208464636,
 'attack_Man City': 1.5943626910700817,
 'attack_Man United': 1.2104512969927286,
 'attack_Newcastle': 0.8429692085737125,
 'attack_Southampton': 0.8814749404469013,
 'attack_Tottenham': 1.2480679730571407,
 'attack_Watford': 1.008878573402672,
 'attack_West Ham': 1.01251649604207,
 'attack_Wolves': 0.9417766288259545,
 'defence_Arsenal': -0.9402296750817191,
 'defence_Bournemouth': -0.6298999824051443,
 'defence_Brighton': -0.8954346287809901,
 'defence_Burnley': -0.7009150478574419,
 'defence_Cardiff': -0.